In [243]:
import cr_analysis as cra
import numpy as np
import pandas as pd
import sklearn.metrics

In [244]:
import imp
imp.reload(cr_analysis)

<module 'cr_analysis' from '/Users/itsnamgyu/code/cardiac-research/cr_analysis.py'>

In [245]:
from cr_analysis import Result

In [226]:
result = Result.select()

--------------------------------- Results List ---------------------------------
  #. KEY                                     ACC       F1_MACRO  
--------------------------------------------------------------------------------
  0. mobileneta25/EXP02_D00_L00_F00          0.7833    0.6459    
  1. mobileneta25/EXP02_D00_L00_F01          0.8327    0.6284    
  2. mobileneta25/EXP02_D00_L00_F02          0.8042    0.5668    
  3. mobileneta25/EXP02_D00_L00_F03          0.7776    0.6380    
  4. mobileneta25/EXP02_D00_L00_F04          0.8308    0.6140    
  5. mobileneta25/EXP02_D00_L01_F00          0.7871    0.4879    
  6. mobileneta25/EXP02_D00_L01_F01          0.4791    0.4743    
  7. mobileneta25/EXP02_D00_L01_F02          0.7167    0.4374    
  8. mobileneta25/EXP02_D00_L01_F03          0.7624    0.4656    
  9. mobileneta25/EXP02_D00_L01_F04          0.7262    0.4145    
 10. mobileneta25/EXP03_D00_L00_F00          0.8783    0.7711    
 11. mobileneta25/EXP03_D00_L00_F01          0

In [218]:
list(result.data['predictions'].values())[0]

{'truth': 'in',
 'prediction': 'in',
 'percentages': {'in': '0.7334755', 'oap': '0.06532377', 'obs': '0.20120081'}}

In [219]:
print(result.describe())

Model             : mobileneta25 analyzed on 2019-05-31 04:29:57
accuracy          : 0.783270
soft_accuracy     : 0.880228
auc_micro         : 0.934393
auc_macro         : 0.906127
f1_micro          : 0.783270
f1_macro          : 0.645866

prediction   in  oap  obs
truth                    
in          341   15   46
oap          17   40   13
obs          22    1   31

     precision    recall
in    0.897368  0.848259
oap   0.714286  0.571429
obs   0.344444  0.574074

          auc        f1
in   0.885392  0.872123
oap  0.956892  0.634921
obs  0.876099  0.430556



In [237]:
result.data['schema_version']

'2.0'

In [246]:
cra.export_csv()

Exporting csv to /Users/itsnamgyu/code/cardiac-research/results.csv


In [180]:
DEFAULT_CLASSES = ['in', 'oap', 'obs']

def _extract_truth_array(prediction, keys=DEFAULT_CLASSES):
    array = []
    for cls in keys:
        if (cls == prediction['truth']):
            array.append(1.0)
        else:
            array.append(0.0)
    return array
            
def _extract_percentage_array(prediction, keys=DEFAULT_CLASSES):
    array = []
    for cls in keys:
        array.append(float(prediction['percentages'][cls]))
    return array

def _extract_prediction_array(prediction, keys=DEFAULT_CLASSES):
    array = []
    for cls in keys:
        if (cls == prediction['prediction']):
            array.append(1.0)
        else:
            array.append(0.0)
    return array

In [158]:
self = result

predictions = self.data['predictions'].values()
truth_vectors = np.array(list(map(_extract_truth_array, predictions)))
percentage_vectors = np.array(list(map(_extract_percentage_array, predictions)))
prediction_vectors = np.array(list(map(_extract_prediction_array, predictions)))

auc = sklearn.metrics.roc_auc_score(truth_vectors, percentage_vectors, average=None)
auc = dict(zip(keys, auc))
f1 = sklearn.metrics.f1_score(truth_vectors, prediction_vectors, average=None)
f1 = dict(zip(keys, f1))

pd.DataFrame(dict(auc=auc, f1=f1))

,auc,f1
in,0.868340,0.880290
oap,0.948371,0.650602
obs,0.889634,0.169492


In [159]:
result.get_auc_f1_by_class()

,auc,f1
in,0.868340,0.880290
oap,0.948371,0.650602
obs,0.889634,0.169492


In [160]:
self = result

predictions = self.data['predictions'].values()
truth_vectors = np.array(list(map(self._extract_truth_array, predictions)))
percentage_vectors = np.array(list(map(self._extract_percentage_array, predictions)))
prediction_vectors = np.array(list(map(self._extract_prediction_array, predictions)))

auc_micro = sklearn.metrics.roc_auc_score(truth_vectors, percentage_vectors, average='micro')
auc_macro = sklearn.metrics.roc_auc_score(truth_vectors, percentage_vectors, average='macro')
f1_micro = sklearn.metrics.f1_score(truth_vectors, prediction_vectors, average='micro')
f1_macro = sklearn.metrics.f1_score(truth_vectors, prediction_vectors, average='macro')

print(auc_micro, auc_macro, f1_micro, f1_macro)

0.9437970767251226 0.9021150059648378 0.8041825095057035 0.5667947135415188


In [181]:
result.get_auc_f1()

{'auc_micro': 0.9343925747083232,
 'auc_macro': 0.9061274590648885,
 'f1_micro': 0.7832699619771865,
 'f1_macro': 0.6458663175415095}